In [ ]:
!pip install openpyxlf
!pip install optuna
!pip install catboost
!pip install category-encoders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.6/390.6 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 1.6 MB/s eta 0:00:00


In [40]:
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from category_encoders.target_encoder import TargetEncoder
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import optuna

ModuleNotFoundError: No module named 'lightgbm'

In [20]:
train = pd.read_csv("final_train.csv")
test = pd.read_csv("final_test.csv")

### Ridge регрессия с L2 регуляризацией

In [21]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3980 entries, 0 to 3979
Data columns (total 47 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   предмет           3980 non-null   object 
 1   ФИО               3980 non-null   object 
 2   tutor_head_tags   3980 non-null   object 
 3   tutor_rating      3980 non-null   float64
 4   tutor_reviews     3980 non-null   int64  
 5   description       3980 non-null   object 
 6   experience_desc   3980 non-null   object 
 7   Education_1       3980 non-null   object 
 8   Desc_Education_1  3980 non-null   object 
 9   Education_2       3980 non-null   object 
 10  Desc_Education_2  3980 non-null   object 
 11  status            3980 non-null   object 
 12  experience        3980 non-null   int64  
 13  categories        3980 non-null   object 
 14  mean_price        3980 non-null   float64
 15  school            3980 non-null   int64  
 16  ege               3980 non-null   int64  


In [23]:
ridge_feat = ['tutor_rating', 'tutor_reviews', 'experience', 'school', 'ege', 'oge', 'geom', 'matan', 'linal',
       'teorver'] + [f'e_{i}' for i in range(24)] 
X_train = train[ridge_feat]
y_train = train['mean_price']
ridge_grid = {'max_iter': [50, 250, 750], 'alpha': [0.02, 2, 20]}
gc_regressor = GridSearchCV(param_grid=ridge_grid,
                            estimator=Ridge())
gc_regressor.fit(X_train, y_train)

GridSearchCV(estimator=Ridge(),
             param_grid={'alpha': [0.02, 2, 20], 'max_iter': [50, 250, 750]})

In [26]:
train[ridge_feat].isna().sum()

tutor_rating     0
tutor_reviews    0
experience       0
school           0
ege              0
oge              0
geom             0
matan            0
linal            0
teorver          0
e_0              0
e_1              0
e_2              0
e_3              0
e_4              0
e_5              0
e_6              0
e_7              0
e_8              0
e_9              0
e_10             0
e_11             0
e_12             0
e_13             0
e_14             0
e_15             0
e_16             0
e_17             0
e_18             0
e_19             0
e_20             0
e_21             0
e_22             0
e_23             0
dtype: int64

In [17]:
sample_submit = pd.read_csv("sample_submit.csv")

In [27]:
sample_submit['mean_price'] =gc_regressor.predict(test[ridge_feat])
sample_submit

,index,mean_price
0,0,17.522450
1,1,12.553556
2,2,18.100676
3,3,12.718804
4,4,12.128113
...,...,...
1511,1511,14.940453
1512,1512,18.776927
1513,1513,14.464385
1514,1514,15.005002


In [28]:
sample_submit.to_csv('ridge_1st.csv', index=None)

### Случайный лес

In [30]:
cat_features = [c for c in test.columns if c not in ridge_feat + ['categories', 'tutor_head_tags']]
X_train = train[ridge_feat + cat_features]
y_train = train['mean_price']
encoder = TargetEncoder(cols=cat_features)
X_train = encoder.fit_transform(X_train, y_train)
forest_grid = {'n_estimators': [250, 500], 'max_depth': [2, 3, 4, 6]}
gc_regressor = GridSearchCV(param_grid=forest_grid,
                            estimator=RandomForestRegressor())
gc_regressor.fit(X_train, y_train)

GridSearchCV(estimator=RandomForestRegressor(),
             param_grid={'max_depth': [2, 3, 4, 6], 'n_estimators': [250, 500]})

In [31]:
forest_test = encoder.transform(test[ridge_feat + cat_features])
sample_submit['mean_price'] = gc_regressor.predict(forest_test)
sample_submit

,index,mean_price
0,0,14.145234
1,1,14.156372
2,2,14.145234
3,3,14.145234
4,4,7.502170
...,...,...
1511,1511,14.145234
1512,1512,14.145234
1513,1513,14.145234
1514,1514,14.145234


In [32]:
sample_submit.to_csv('forest_1st.csv', index=None)

In [33]:
train.head()

,предмет,ФИО,tutor_head_tags,tutor_rating,tutor_reviews,description,experience_desc,Education_1,Desc_Education_1,Education_2,...,e_14,e_15,e_16,e_17,e_18,e_19,e_20,e_21,e_22,e_23
0,matematika,Репетитор по математике Назарова Анастасия Ан...,"['Школьная программа', 'Подготовка к ЕГЭ', 'По...",4.0,7,Репетитор не предоставил о себе дополн...,неизвестно,Вологодский государственный педагогический ун...,"Год окончания: 2017, Факультет: факультет при...",Череповецкий государственный университет,...,0.015016,-0.041920,0.032214,-0.033423,-0.039425,-0.020654,-0.057637,0.009328,-0.060357,-0.029576
1,informatika,Репетитор по информатике Мотроненко Инна Конст...,"['Школьная программа', 'Подготовка к ЕГЭ', 'По...",0.0,0,"Выезд за город: Балашиха-1, Балашиха-2...",Московская Школа (учитель ...,Московский городской педагогический университет,"Год окончания: 2008, Факультет: Математически...",Московский городской педагогический университ...,...,0.075222,-0.009125,-0.045767,0.027953,0.034945,0.015114,0.037616,0.026814,-0.056034,0.126616
2,matematika,Репетитор по математике Логинова Елизавета Ал...,"['Школьная программа', 'Математический анализ'...",4.0,19,"Здравствуйте! Меня зовут Елизавета, о...",неизвестно,Санкт-Петербургский государственный университет,"Год окончания: 2021, Факультет: физический фа...",неизвестно,...,0.003656,0.042084,-0.029320,0.009124,-0.010459,-0.004905,-0.008889,-0.037009,-0.026889,0.020685
3,matematika,Репетитор по математике Харланов Олег Викторо...,"['Школьная программа', 'Подготовка к ЕГЭ', 'По...",0.0,0,"Целеустремленный, ориентирован на резу...",преподавателем ...,Приазовский государственный технический униве...,"Год окончания: 2005, Факультет: Энергетически...",неизвестно,...,-0.004428,0.044438,0.040700,-0.004475,0.006056,0.033277,-0.027352,0.008591,0.015483,-0.009675
4,matematika,Репетитор по математике Позябин Дмитрий Евген...,"['Школьная программа', 'Подготовка к ЕГЭ', 'По...",0.0,0,Репетитор не предоставил о себе дополн...,неизвестно,Московский государственный технический универ...,"Год окончания: 2019, Факультет: факультет эне...",неизвестно,...,0.047266,-0.011640,-0.016769,-0.046544,-0.016477,0.022656,-0.027924,0.038961,0.030202,0.018404


### Градиентный бустинг

In [34]:
cat_features = [c for c in test.columns if c not in ridge_feat + ['categories', 'tutor_head_tags']]
X_train = train[ridge_feat + cat_features]
y_train = train['mean_price']

In [35]:
def objective(trial):
    train_x, valid_x, train_y, valid_y = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

    param = {
        "learning_rate": trial.suggest_float("learning_rate", 0.003, 0.3),
        "max_depth": trial.suggest_int("max_depth", 2, 6),
    }
    cat = CatBoostRegressor(cat_features=cat_features, **param)

    cat.fit(train_x, train_y, eval_set=[(valid_x, valid_y)], verbose=False, early_stopping_rounds=20)

    preds = cat.predict(valid_x)
    rmse = mean_squared_error(valid_y, preds, squared=False)
    return rmse

In [36]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=17)
trial = study.best_trial

[I 2023-07-29 00:55:30,311] A new study created in memory with name: no-name-d3cc5777-c64d-4740-bfe9-6bc8676be328
[I 2023-07-29 00:55:32,009] Trial 0 finished with value: 7.371101950858216 and parameters: {'learning_rate': 0.038572335516517633, 'max_depth': 5}. Best is trial 0 with value: 7.371101950858216.
[I 2023-07-29 00:55:32,616] Trial 1 finished with value: 7.389844492452667 and parameters: {'learning_rate': 0.09251876393876454, 'max_depth': 6}. Best is trial 0 with value: 7.371101950858216.
[I 2023-07-29 00:55:33,214] Trial 2 finished with value: 7.398381002277493 and parameters: {'learning_rate': 0.21798273848182978, 'max_depth': 6}. Best is trial 0 with value: 7.371101950858216.
[I 2023-07-29 00:55:34,387] Trial 3 finished with value: 7.391567874534648 and parameters: {'learning_rate': 0.038431521261696154, 'max_depth': 4}. Best is trial 0 with value: 7.371101950858216.
[I 2023-07-29 00:55:34,713] Trial 4 finished with value: 7.427421616539741 and parameters: {'learning_rate':

In [37]:
best_cat = CatBoostRegressor(cat_features=cat_features, **study.best_trial.params)
train_x, valid_x, train_y, valid_y = train_test_split(X_train, y_train, test_size=0.25, random_state=0)
best_cat.fit(train_x, train_y, eval_set=[(valid_x, valid_y)], verbose=0, early_stopping_rounds=20)
n_estimators = best_cat.best_iteration_
final_cat = CatBoostRegressor(cat_features=cat_features, iterations=n_estimators, **trial.params)
final_cat.fit(X_train, y_train)

0:	learn: 8.2841143	total: 4.43ms	remaining: 1.28s
1:	learn: 8.2504492	total: 7.97ms	remaining: 1.15s
2:	learn: 8.2150845	total: 10.4ms	remaining: 995ms
3:	learn: 8.1849209	total: 14.2ms	remaining: 1.01s
4:	learn: 8.1613894	total: 17.6ms	remaining: 1s
5:	learn: 8.1338440	total: 21.1ms	remaining: 998ms
6:	learn: 8.1043782	total: 24.8ms	remaining: 1s
7:	learn: 8.0793462	total: 28.1ms	remaining: 992ms
8:	learn: 8.0539805	total: 32ms	remaining: 998ms
9:	learn: 8.0283793	total: 35.5ms	remaining: 995ms
10:	learn: 8.0108345	total: 39.3ms	remaining: 997ms
11:	learn: 7.9905909	total: 43.1ms	remaining: 999ms
12:	learn: 7.9682369	total: 46.2ms	remaining: 984ms
13:	learn: 7.9505933	total: 49.9ms	remaining: 983ms
14:	learn: 7.9326691	total: 53.3ms	remaining: 977ms
15:	learn: 7.9140276	total: 56.6ms	remaining: 970ms
16:	learn: 7.8940139	total: 60.9ms	remaining: 978ms
17:	learn: 7.8740563	total: 64.1ms	remaining: 968ms
18:	learn: 7.8561716	total: 67.9ms	remaining: 968ms
19:	learn: 7.8434825	total: 71

In [38]:
sample_submit['mean_price'] = final_cat.predict(test[ridge_feat + cat_features])
sample_submit

,index,mean_price
0,0,16.983295
1,1,11.978964
2,2,18.253270
3,3,12.644527
4,4,8.738107
...,...,...
1511,1511,12.671122
1512,1512,17.909139
1513,1513,16.120196
1514,1514,14.748191


In [39]:
sample_submit.to_csv('boosting_1st.csv', index=None)